# Genre Fix

This dataset has too many genres, they are too granular. The aim of this notebook:
1. Split problematic genres, those with "joint" genres, e.g. "Science and Drama".
2. Normalize genres to common names.
3. Combine genres until there are 10 main ones.

## Import data

In [1]:
import os
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

notebook_dir = Path().resolve()
base_path = os.path.abspath(notebook_dir.parent.parent)
sys.path.append(base_path)
from src.data_utils import *

In [2]:
embeddings, movie_ids  = load_movie_embeddings(os.path.join(base_path, "data", "data_final"))

In [3]:
embeddings.shape

(161553, 1024)

In [4]:
movie_ids.shape

(161553,)

In [5]:
movie_df = load_movie_data(os.path.join(base_path, "data", "data_final"))

In [6]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213903 entries, 0 to 213902
Data columns (total 22 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   movie_id        213903 non-null  object
 1   title           213903 non-null  object
 2   summary         0 non-null       object
 3   release_date    213903 non-null  object
 4   genre           176075 non-null  object
 5   director        190332 non-null  object
 6   actors          145306 non-null  object
 7   duration        131113 non-null  object
 8   imdb_id         201370 non-null  object
 9   country         204335 non-null  object
 10  sitelinks       213903 non-null  object
 11  wikipedia_link  213903 non-null  object
 12  budget          8609 non-null    object
 13  box_office      7786 non-null    object
 14  awards          9830 non-null    object
 15  set_in_period   3500 non-null    object
 16  year            213903 non-null  int64 
 17  popularity      181485 non-nu

In [7]:
movie_df[movie_df.year==1950].shape

(1523, 22)

## Genre exploration

In [8]:
all_genres = movie_df[movie_df.genre.notna()].genre.unique()

In [9]:
# Sample randomly and see examples of genres
import random

print(random.choice(all_genres))
print(random.choice(all_genres))
print(random.choice(all_genres))
print(random.choice(all_genres))
print(random.choice(all_genres))

horror film, zombie comedy
pop music, musical film, vampire film
drama film, action film, war film, film adaptation, thriller film
comedy film, adventure film, film based on a novel
road movie, comedy drama, LGBT-related film


In [10]:
random.choice(all_genres[np.array(["and" in i for i in all_genres])])

'documentary film, silent film, propaganda film'

It seems like the common delimeters are commas and "and". Splitting by these should give us all possible genres, however there weirder ones like "film from a novel". 

In [11]:
# First split by "," and "and"
import re

new_genres = []
for row in all_genres:
    split_genres = row.split(",")
    split_genres = [i.lower().replace("film", "").strip() for i in split_genres]
    new_genres.extend(split_genres)

In [12]:
from collections import Counter

In [13]:
genre_counter = Counter(new_genres)

In [14]:
len(genre_counter)

975

In [16]:
genre_counter.most_common(20)

[('drama', 4274),
 ('action', 2416),
 ('comedy', 2388),
 ('thriller', 1783),
 ('science fiction', 1655),
 ('horror', 1632),
 ('fantasy', 1623),
 ('adventure', 1619),
 ('crime', 1394),
 ('romance', 1073),
 ('based on a novel', 1045),
 ('mystery', 883),
 ('musical', 831),
 ('lgbt-related', 822),
 ('comedy drama', 742),
 ('based on literature', 728),
 ('biographical', 674),
 ('family', 656),
 ('teen', 612),
 ('romantic comedy', 595)]

Use sentence transformers to embed genres and cluster them.

In [18]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

/Users/nselcheung/opt/anaconda3/envs/data_literacy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [20]:
genre_embeddings = model.encode(new_genres)

In [21]:
genre_embeddings.shape

(47655, 384)

In [22]:
# Clustering
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=123)
labels = kmeans.fit_predict(genre_embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/nselcheung/opt/anaconda3/envs/data_literacy/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/nselcheung/opt/anaconda3/envs/data_literacy/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/nselcheung/opt/anaconda3/envs/data_literacy/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/nselcheung/opt/anaconda3/envs/data_literacy/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:237: RuntimeWarning: divide by zero encounter

In [23]:
genre_clusters = pd.DataFrame({"genre": new_genres, "cluster": labels})

In [42]:
genre_clusters[genre_clusters.cluster==9].genre.unique()

array(['noir', 'thriller', 'suspense', 'crime thriller',
       'political thriller', 'horror', 'psychological thriller', 'ghost',
       'comedy thriller', 'romantic thriller', 'vampire', 'zombie',
       'supernatural horror', 'science fiction horror', 'horror fiction',
       'psychological horror', 'natural horror', 'neo-noir',
       'action thriller', 'erotic thriller', 'midnight movie',
       'animal horror', 'supernatural', 'horror western', 'body horror',
       'horror fantasy', 'gothic horror', 'thriller anime',
       'erotic horror', 'list of holiday horror s',
       'horror anime and manga', 'thriller television series',
       'horror television series', 'techno-thriller', 'legal thriller',
       'body horror television program', 'lovecraftian horror',
       'japanese horror', 'medical thriller', 'horrorcore', 'halloween',
       'horror literature', 'cosmic horror', 'teen horror',
       'financial thriller', 'paranormal fiction', 'folk horror',
       'thriller pla

10 Main clusters:
* cluster 0: Others
* cluster 1: Comedy
* cluster 2: Romance
* cluster 3: Drama
* cluster 4: Fantasy
* cluster 5: Action
* cluster 6: Science Fiction
* cluster 7: Family
* cluster 8: Mystery
* cluster 9: Thriller

In [43]:
cluster_label_mapping = {
    0: "others",
    1: "comedy",
    2: "romance",
    3: "drama",
    4: "fantasy",
    5: "action",
    6: "scifi",
    7: "family",
    8: "mystery",
    9: "thriller"
}

In [44]:
genre_clusters["new_label"] = genre_clusters["cluster"].apply(lambda x: cluster_label_mapping[x])

In [46]:
genre_clusters

,genre,cluster,new_label
0,musical,1,comedy
1,fantasy,4,fantasy
2,melodrama,0,others
3,musical,1,comedy
4,romance,2,romance
...,...,...,...
47650,prison,1,comedy
47651,musical,1,comedy
47652,lgbt-related,1,comedy
47653,thriller,9,thriller


In [52]:
for genre in cluster_label_mapping.values():
    print(f"Genre - {genre.upper()}")
    original_genres = genre_clusters[genre_clusters.new_label == genre].genre.unique()
    for i, original_g in enumerate(original_genres):
        print(f"    - {original_g}")
        if i == 7: 
            break

Genre - OTHERS
    - melodrama
    - cinematic fairy tale
    - samurai cinema
    - medieval
    - flashback
    - heist
    - gangster
    - western
Genre - COMEDY
    - musical
    - comedy
    - crime
    - comedy drama
    - prison
    - crime drama
    - lgbt-related
    - comedy horror
Genre - ROMANCE
    - romance
    - romantic comedy
    - erotic
    - romantic drama
    - sex
    - erotica
    - lesbian pornographic
    - romantic fantasy
Genre - DRAMA
    - drama
    - historical drama
    - psychological drama
    - period drama
    - spoken drama
    - war drama
    - political drama
    - drama television series
Genre - FANTASY
    - fantasy
    - adventure
    - adventure fiction
    - fantasy anime and manga
    - adventure anime and manga
    - action-adventure
    - space adventure
    - adventure television series
Genre - ACTION
    - action
    - live action
    - action anime
    - action game
Genre - SCIFI
    - based on literature
    - based on a novel
    - bi